In [15]:
!pip install tabpfn -q

In [16]:
import pandas as pd
from tabpfn import TabPFNClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

In [17]:
rent_df = pd.read_csv('after_transformations.csv')

In [18]:

num_buckets = 8

# Create quantile-based buckets with integer labels
rent_df['rent_bucket'] = pd.qcut(rent_df['price'], q=num_buckets, labels=[i for i in range(num_buckets)])

# Display the bins and distribution
bin_edges = pd.qcut(rent_df['price'], q=num_buckets, retbins=True)[1]
distribution = rent_df['rent_bucket'].value_counts(normalize=True, sort=False)

label_to_range: dict[int, str] = {i: f"${bin_edges[i]:,.2f} - ${bin_edges[i+1]:,.2f}" for i in range(len(bin_edges) - 1)}

for i in range(len(bin_edges) - 1):
    print(f"Bucket {i+1}: ${bin_edges[i]:,.2f} - ${bin_edges[i+1]:,.2f} with {round(distribution.iloc[i]*100, ndigits=2)}% of data")

Bucket 1: $7.09 - $7.72 with 12.51% of data
Bucket 2: $7.72 - $7.88 with 12.49% of data
Bucket 3: $7.88 - $8.01 with 13.3% of data
Bucket 4: $8.01 - $8.10 with 11.79% of data
Bucket 5: $8.10 - $8.22 with 13.43% of data
Bucket 6: $8.22 - $8.38 with 11.71% of data
Bucket 7: $8.38 - $8.74 with 12.38% of data
Bucket 8: $8.74 - $11.74 with 12.39% of data


In [19]:
X = rent_df.drop(columns=['price', 'rent_bucket'])
y = rent_df['rent_bucket']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

In [ ]:
classifier.fit(X_train, y_train, overwrite_warning=True)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

print('Accuracy', accuracy_score(y_test, y_eval))

/Users/utkarsh/Desktop/Utkarsh/NYU/Year 2/Semester 1/DS/Project/rent-raja/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/utkarsh/Desktop/Utkarsh/NYU/Year 2/Semester 1/DS/Project/rent-raja/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/utkarsh/Desktop/Utkarsh/NYU/Year 2/Semester 1/DS/Project/rent-raja/.venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on t